In [ ]:
!pip install ensemble_boxes

In [2]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [18]:
# ensemble csv files
# submission_files = ['/data/ephemeral/home/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_trash/20240112_144452/submit/submission.csv',
#                     '/data/ephemeral/home/mmdetection/work_dirs/cascade-rcnn_r50-caffe_fpn_1x_coco/20240112_185709/submit/submission.csv',
#                     '/data/ephemeral/home/mmdetection/work_dirs/cascade-rcnn_r50-caffe_fpn_1x_coco/20240112_204451/submit/submission.csv',
#                     '/data/ephemeral/home/mmdetection/work_dirs/cascade-rcnn_r50-caffe_fpn_1x_coco/20240113_000650/submit/submission.csv',
#                    '/data/ephemeral/home/mmdetection/work_dirs/cascade-rcnn_r50-caffe_fpn_1x_coco/20240113_013843/submit/submission.csv']
submission_files = ['/data/ephemeral/home/mmdetection/work_dirs/cascade-rcnn_x101-64x4d_fpn_1x_coco.py/20240114_190924/submit/submission_data4.csv',
                    '/data/ephemeral/home/mmdetection/work_dirs/cascade-rcnn_x101-64x4d_fpn_1x_coco.py/20240114_143934/submit/submission_data3.csv']
submission_df = [pd.read_csv(file) for file in submission_files]

In [19]:
image_ids = submission_df[0]['image_id'].tolist()

In [20]:
# ensemble 할 file의 image 정보를 불러오기 위한 json
annotation = '/data/ephemeral/home/dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [21]:
prediction_strings = []
file_names = []
# ensemble 시 설정할 iou threshold 이 부분을 바꿔가며 대회 metric에 알맞게 적용해봐요!
iou_thr = 0.5

# 각 image id 별로 submission file에서 box좌표 추출
for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []

    index_str = image_id.split('/')[1].split('.')[0]
    image_index = int(index_str)
    
    image_info = coco.loadImgs(image_index)[0]
#     각 submission file 별로 prediction box좌표 불러오기
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()

        if len(predict_list)==0 or len(predict_list)==1:
            continue

        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []

        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)

        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))

#     예측 box가 있다면 이를 ensemble 수행
    if len(boxes_list):
        boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(label) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '

    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [22]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('ensemble/submission_ensemble_data_cascade34_101.csv', index=False)

submission.head()

,PredictionString,image_id
0,0 0.7592291831970215 215.37314 692.6747 250.07...,test/0000.jpg
1,0 0.9471516013145447 762.1887 663.8485 895.771...,test/0001.jpg
2,0 0.9506085515022278 658.9348 288.84018 736.65...,test/0002.jpg
3,0 0.06530828028917313 2.9803123 0.0 990.2394 1...,test/0003.jpg
4,0 0.8522250652313232 194.36028 259.7238 871.43...,test/0004.jpg
